# Miniature Neural Network Example

This notebook implements a neural network that, when sufficiently trained, can execute exclusive or (XOR) decisions.

This is based on [the implementation by Konstantinos Kitsios](https://towardsdatascience.com/how-to-build-a-simple-neural-network-from-scratch-with-python-9f011896d2f3), with [source code on GitLab](https://gitlab.com/kitsiosk/xor-neural-net/tree/master).

## Import libraries

First, we'll be working with numerical matrices. Since these data structures are not part of the base collection of Python 3 libraries, we'll need to import a new library `numpy` to take care of that. When we import, we're providing an alias `np` so that we can use this to refer to that library in an abbreviated manner in future code.

In [3]:
import numpy as np

Let's look at this library a bit.

## Intro to matrices in NumPy

Matrices are mathematical structures represented as a grid of numbers, for example:

$$\begin{bmatrix} 
1 & 2 & 3 \\
4 & 5 & 6
\end{bmatrix}$$

In NumPy, matrices are represented as arrays filled with arrays of numbers, each representing a row of the matrix.

We can create matrices filled with random numbers, or with ones or zeros easily using NumPy's convenience functions.

In [3]:
np.random.randn(3, 2)

array([[-0.43773836,  0.30405338],
       [-0.12310372,  1.62265474],
       [ 1.07894756,  0.36820657]])

In [33]:
np.random.randn(5, 5)

array([[ 0.69946656, -1.02740829, -0.24420973, -0.2411146 , -0.61887109],
       [-0.4944247 ,  0.56966557,  1.07035792, -1.14609221,  0.69111952],
       [ 0.82643473,  0.92277872, -0.25948179,  0.72127418,  1.56117944],
       [-0.25629474,  0.65354462, -0.30536742, -0.96844837, -0.15398082],
       [-0.56258033,  0.07955386, -1.59653425, -0.22107714,  1.68083691]])

In [34]:
np.ones((3,2))

array([[1., 1.],
       [1., 1.],
       [1., 1.]])

In [7]:
np.zeros((4, 3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

We can also multiply matrices, either as a dot-product or a cross-product.

In [28]:
A = np.random.randn(3, 3)
B = np.random.randn(3, 2)

In [29]:
A

array([[ 0.14204565, -0.70993346, -0.84807117],
       [ 0.0107026 , -0.46219455,  1.48010672],
       [ 0.45563562,  1.22179254,  0.03044326]])

In [30]:
B

array([[ 0.18889061, -0.25707058],
       [ 1.24349507, -0.34735751],
       [ 0.98607691, -0.4372636 ]])

In [31]:
np.cross(A, B)

array([[-0.21801415, -0.16019268,  0.09758401],
       [ 0.51412619,  1.84050541,  0.57101901],
       [ 0.01331173,  0.0300194 , -1.40401428]])

In [32]:
np.dot(A, B)

array([[-1.69223107,  0.58091562],
       [ 0.88678405, -0.48940137],
       [ 1.63537768, -0.55484106]])

There are some other convenient functions for creating and manipulating matrices as well, which you can read about in the NumPy documentation: https://docs.scipy.org/doc/

## Creating our Neural Network

### The Goal

To start, let's define the problem we want our neural network to solve.

Given two inputs $a$ and $b$, we would like our network to decide the result $c$ as follows:

$$\frac{\left.\begin{matrix}
a & b &
\end{matrix}\right|\begin{matrix}
& \hat{y}
\end{matrix}}{\left.\begin{matrix}
0 & 0 & \\
0 & 1 & \\
1 & 0 & \\
1 & 1 & 
\end{matrix}\right|\begin{matrix}
& 0 \\ & 1  \\ & 1 \\ & 0
\end{matrix}}$$

Neural networks work by passing inputs through successive layers of artificial neurons. The inputs are analogous to neurotransmitter activations of biological neurons, and the neurons apply a transformation function to the inputs when activated.

In practice, neural networks can have many millions of neurons in many hundreds or thousands of layers, with most of those layers hidden from inspection.

For this example, we'll use a simple 3-layer neural network: an input layer ($X$ in the diagram below) with only our two inputs $a$ and $b$, a single hidden layer ($A1$ in the diagram), and the output layer ($A2$ in the diagram) with our output $\hat{y}$.

![Neural network diagram](https://cdn-images-1.medium.com/max/1600/1*dFbZFuwqoR2YIHS4YcC9Jg.jpeg)

You'll also notice in the diagram that we've defined transformation information along the edges moving from one layer to the next. These choices are important, as they define the behavior of neural firing in the network.

For our activation functions (simulating the activation of a neuron by neurotransmitter stimulus), we're using the hyperbolic tangent (which we'll call $h$) and sigmoid (which we'll call $g$) functions. The sigmoid function is a good choice for the last layer in this network because it outputs values between 0 and 1, with a relatively sharp transition from one to the other. The hyperbolic tangent function is a reasonable choice for the hidden layer, but many other functions could work here. So, our activation functions are:

$$\begin{align}
h(z) &= \tanh(x) \\
g(z) &= \frac{1}{1 + e^{-z}}
\end{align}$$


When we calculate the "activated" value sent to the next layer, these are computed as follows:

$$\begin{align}
A1 &= h(W1\cdot X + b1) \\
A2 &= g(W2\cdot A1 + b2)
\end{align}$$

The values $W1$ and $W2$ are called "weights", and the values $b1$ and $b2$ are called "biases". Together, these weights and biases make up the parameters to be learned by the neural network. These values are generally matrices.

To start, let's implement the sigmoid activation function.

In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# This computes on matrices, but let's check some single values to get a feel for it.
print(f"{'z':>6s} | {'g(z)':>18s}")
for z in range(-10, 10):
    print(f"{z:>6d} | {sigmoid(z):>18.16f}")


     z |               g(z)
   -10 | 0.0000453978687024
    -9 | 0.0001233945759862
    -8 | 0.0003353501304665
    -7 | 0.0009110511944006
    -6 | 0.0024726231566348
    -5 | 0.0066928509242849
    -4 | 0.0179862099620916
    -3 | 0.0474258731775668
    -2 | 0.1192029220221175
    -1 | 0.2689414213699951
     0 | 0.5000000000000000
     1 | 0.7310585786300049
     2 | 0.8807970779778823
     3 | 0.9525741268224334
     4 | 0.9820137900379085
     5 | 0.9933071490757153
     6 | 0.9975273768433653
     7 | 0.9990889488055994
     8 | 0.9996646498695336
     9 | 0.9998766054240137


Next, we'll initialize our parameter values based on the number of neurons in each layer of the neural network. Our weights need to be initialized with random values from a normal distribution, and our biases start at zero.

In [5]:
def init_parameters(num_inputs, num_hidden, num_outputs):
    # Weights (np.random.randn returns random normally distributed matrices)
    W1 = np.random.randn(num_hidden, num_inputs)
    W2 = np.random.randn(num_outputs, num_hidden)
    # Biases (np.zeros returns matrices with the specified dimensions with all elements zero)
    b1 = np.zeros((num_hidden, 1))
    b2 = np.zeros((num_outputs, 1))
    # Create and return our params dictionary
    params = {
        'W1': W1,
        'b1': b1,
        'W2': W2,
        'b2': b2
    }
    return params

print(init_parameters(2, 2, 1))
print(init_parameters(4, 3, 2))

{'W1': array([[-0.89857283,  0.79062035],
       [-1.26652309,  0.57572595]]), 'b1': array([[0.],
       [0.]]), 'W2': array([[-0.41978651, -0.03815815]]), 'b2': array([[0.]])}
{'W1': array([[ 0.30117773, -0.83138512,  0.56300765, -0.15833648],
       [-1.01901354, -1.63416825, -0.23911938,  1.35304578],
       [ 0.53346752,  0.25131605, -0.8339447 , -0.49249342]]), 'b1': array([[0.],
       [0.],
       [0.]]), 'W2': array([[ 0.31271814, -0.29261628,  0.67403643],
       [ 0.50532101,  2.58823728, -1.606086  ]]), 'b2': array([[0.],
       [0.]])}


Now, it's time to create the function that will propagate activations forward through the network.

In [19]:
def forward_prop(inputs, params):
    # Take apart arguments and put them in math notation
    X = inputs
    W1 = params['W1']
    b1 = params['b1']
    W2 = params['W2']
    b2 = params['b2']
    # Propagate the inputs into the hidden layer
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    # Propagate the hidden layer into the output layer
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    # Remember the values of A1 and A2 for next time
    cache = {
        'A1': A1,
        'A2': A2
    }
    # Return a tuple of the result and the cache
    return A2, cache

# Let's try this out...
p = init_parameters(2,2,1) # Set our parameters
X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]]) # The 4 training examples by columns
print(X)
Y_hat, cache = forward_prop(X, p)
print(Y_hat)

[[0 0 1 1]
 [0 1 0 1]]
[[0.5        0.38686082 0.50245512 0.38807735]]


Since we've now calculated a result of the network, we can also compute the loss function. This function tells us how far from the correct the network's prediction was. For this, we're using a calculation method called the "Cross Entropy Loss Function".

In [23]:
def calculate_cost(prediction, truth, num_training_examples):
    A2 = prediction
    Y = truth
    m = num_training_examples
    cost = -np.sum(np.multiply(Y, np.log(A2)) + np.multiply(1-Y, np.log(1-A2)))/m
    cost = np.squeeze(cost)
    return cost

# Let's get a sense for this function:
Y = np.array([[0, 1, 1, 0]]) # ground truth
m = X.shape[1] # number of training examples
# Start with something really dumb
Y_hat = np.array([[0.5, 0.5, 0.5, 0.5]])
print(calculate_cost(Y_hat, Y, m))

0.6931471805599453


Now for the learning! Neural networks also propagate information backward in order to learn better performance. In our case, we'll be using a method called "Gradient Descent" to train our neural network. The input for that method will be the gradients of the loss function we just implemented at the values of our parameters (weights and biases).

A gradient of a function at a specific point is the slope of a line in a particular direction on the surface defined by that function. You can think of the gradient in the x direction of a function as a measure of the steepness of the surface when looking in the x-axis direction.

We'll be taking our gradients in the directions of the weights $W1$ and $W2$.

Note: This is the toughest part of a neural network algorithm, so if this piece doesn't make sense to you, don't worry too much about it. This will become much clearer after a bit of multivariable calculus.

Let's go ahead and implement a function that performs the backward propagation.

In [27]:
def back_prop(inputs, truth, cache, params, num_training_examples):
    # Handle arguments
    X = inputs
    Y = truth
    A1 = cache['A1']
    A2 = cache['A2']
    W2 = params['W2']
    m = num_training_examples
    # Compute gradients
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T)
    db2 = np.sum(dZ2, axis=1, keepdims=True)/m
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1-np.power(A1, 2))
    dW1 = np.dot(dZ1, X.T)/m
    db1 = np.sum(dZ1, axis=1, keepdims=True)/m
    # Return gradients
    return {
        'dW1': dW1,
        'db1': db1,
        'dW2': dW2,
        'db2': db2
    }

# Let's test this out.
gradients = back_prop(X, Y, cache, p, m)
print(gradients)

{'dW1': array([[ 0.0511386 ,  0.05107287],
       [-0.00272696, -0.00308779]]), 'db1': array([[ 0.05038723],
       [-0.00230877]]), 'dW2': array([[-0.09889518, -0.20508841]]), 'db2': array([[-0.05565168]])}


With our gradient calculations done, we can move on to implementing the rest of the Gradient Descent method: the updates to our parameters. This is the piece we refer to when we say that a neural network "learns".

To implement this, we need a new "hyperparameter" -- this defines the learning/training mechanism. This parameter is the learning rate, and it controls how much the Gradient Descent step can change the parameters it is learning at each iteration of the algorithm.

Let's implement a function to update the parameters.

In [30]:
def update_params(params, gradients, learning_rate):
    W1 = params['W1'] - learning_rate * gradients['dW1']
    b1 = params['b1'] - learning_rate * gradients['db1']
    W2 = params['W2'] - learning_rate * gradients['dW2']
    b2 = params['b2'] - learning_rate * gradients['db2']
    new_params = {
        'W1': W1,
        'b1': b1,
        'W2': W2,
        'b2': b2
    }
    return new_params

# Let's try it out
for learning_rate in [0.1 * x for x in range(0, 5)]:
    new_params = update_params(p, gradients, learning_rate)
    print(p, new_params)


{'W1': array([[0.00580873, 0.46564667],
       [0.39048838, 0.54514684]]), 'b1': array([[0.],
       [0.]]), 'W2': array([[-1.10948328,  0.04374929]]), 'b2': array([[0.]])} {'W1': array([[0.00580873, 0.46564667],
       [0.39048838, 0.54514684]]), 'b1': array([[0.],
       [0.]]), 'W2': array([[-1.10948328,  0.04374929]]), 'b2': array([[0.]])}
{'W1': array([[0.00580873, 0.46564667],
       [0.39048838, 0.54514684]]), 'b1': array([[0.],
       [0.]]), 'W2': array([[-1.10948328,  0.04374929]]), 'b2': array([[0.]])} {'W1': array([[0.00069487, 0.46053938],
       [0.39076108, 0.54545562]]), 'b1': array([[-0.00503872],
       [ 0.00023088]]), 'W2': array([[-1.09959377,  0.06425813]]), 'b2': array([[0.00556517]])}
{'W1': array([[0.00580873, 0.46564667],
       [0.39048838, 0.54514684]]), 'b1': array([[0.],
       [0.]]), 'W2': array([[-1.10948328,  0.04374929]]), 'b2': array([[0.]])} {'W1': array([[-0.00441899,  0.45543209],
       [ 0.39103377,  0.5457644 ]]), 'b1': array([[-0.01007745],
  

Now we can put all this together into a new function that iterates a certain number of times, or until our cost is low enough.

In [40]:
def train_model(inputs, truth, num_inputs, num_hidden, num_outputs, num_iterations, learning_rate, num_training_examples):
    X = inputs
    Y = truth
    params = init_parameters(num_inputs, num_hidden, num_outputs)
    for k in range(0, num_iterations + 1):
        Y_hat, cache = forward_prop(X, params)
        cost = calculate_cost(Y_hat, Y, num_training_examples)
        gradients = back_prop(X, Y, cache, params, num_training_examples)
        params = update_params(params, gradients, learning_rate)
        if (k % 100 == 0):
            print(f"Cost after iteration # {k:d}: {cost:f}")
    return params

We'll try to train something in a minute -- first, let's get the other side of this coin finished: the ability to predict from the neural network. We'll predict by setting a threshold of 0.5, applying the trained parameters to our inputs, and returning the guessed result.

In [32]:
def predict(inputs, params):
    Y_hat, cache = forward_prop(inputs, params)
    Y_hat = np.squeeze(Y_hat)
    if (Y_hat >= 0.5):
        return 1
    else:
        return 0

With all these functions implemented, we're finally ready to train a neural network and get a prediction! Here we go!

In [42]:
# Reset the random number seed
np.random.seed(2)

# Setup our 4 training examples by column
X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])

# Our ground truth: the outputs of the XOR for every example in X
Y = np.array([[0, 1, 1, 0]])

# The number of training examples
num_training_examples = X.shape[1]

# The hyper-parameters defining the neural model and learning characteristics
num_input_neurons = 2
num_hidden_neurons = 2
num_output_neurons = 1
num_iterations = 1000
learning_rate = 0.3

# Train the neural network
trained_params = train_model(X, Y, num_input_neurons, num_hidden_neurons, num_output_neurons,
                             num_iterations, learning_rate, num_training_examples)

# Get a prediction! We can try any of these: (0, 0), (0, 1), (1, 0). (1, 1)
for x1, x2 in [(a, b) for a in range(2) for b in range(2)]:
    X_test = np.array([[x1], [x2]])
    Y_predict = predict(X_test, trained_params)
    print(f"Neural network prediction for inputs ({X_test[0][0]}, {X_test[1][0]}) is {Y_predict}")

Cost after iteration # 0: 0.856267
Cost after iteration # 100: 0.282135
Cost after iteration # 200: 0.072281
Cost after iteration # 300: 0.031606
Cost after iteration # 400: 0.019622
Cost after iteration # 500: 0.014160
Cost after iteration # 600: 0.011064
Cost after iteration # 700: 0.009075
Cost after iteration # 800: 0.007692
Cost after iteration # 900: 0.006675
Cost after iteration # 1000: 0.005895
Neural network prediction for inputs (0, 0) is 0
Neural network prediction for inputs (0, 1) is 1
Neural network prediction for inputs (1, 0) is 1
Neural network prediction for inputs (1, 1) is 0


And that's it! We've built a neural network from scratch, albeit a simple one!

Here are some ways to further explore this:

Try modifying the hyperparameters.

* What happens if you change the number of neurons in a layer?
* What happens if you modify the learning rate?
* What happens if you change the number of iterations?
* Can you make a prediction fail just by changing hyperparameters?

Generalize this model.

* This model currently executes an XOR operation on two inputs. How would you extend this to a mutual XOR on multiple inputs?

Can you think of other ways to modify what's done here?